In [56]:
# import MongoDB Client & necessary packages
from pymongo import MongoClient
import pprint
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Requests sends and recieves HTTP requests
import requests

# Beautiful Soup parses HTML documents in python
from bs4 import BeautifulSoup

import json
import time

In [57]:
# Search Page Number 1 - 200 = good to go
api_url = 'https://www.goodreads.com/search?page=1&q=Computer+Science&qid=iHFbTUVsHL&search_type=books&tab=books&utf8=%E2%9C%93'
r = requests.get(api_url)
r.status_code

200

In [58]:
# simple print raw text
import pprint
pprint.pprint(r.text[:1000])

('<!DOCTYPE html>\n'
 '<html class="desktop\n'
 '">\n'
 '<head>\n'
 '  <title>Search results for &quot;Computer Science&quot; (showing 1-20 of '
 '8797 books)</title>\n'
 '\n'
 "<meta content='Search results for Computer Science: , , , , , , , , , , , , "
 ", , , , , , , and ' name='description'>\n"
 "<meta content='telephone=no' name='format-detection'>\n"
 "<meta content='noindex' name='robots'>\n"
 '\n'
 '\n'
 '\n'
 '    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();\n'
 ' </script>\n'
 '  <script type="text/javascript">\n'
 '    var ue_mid = "A1PQBFHBHS6YH1";\n'
 '    var ue_sn = "www.goodreads.com";\n'
 '    var ue_furl = "fls-na.amazon.com";\n'
 '    var ue_sid = "131-6860369-3136922";\n'
 '    var ue_id = "WCQD8Q4RDAQDW257E55H";\n'
 '\n'
 '    (function(e){var c=e;var '
 'a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new '
 'Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function '
 'd(h){return function(){a.q.push({n:h,a:arguments,

In [59]:
# parses HTML documents in python
soup = BeautifulSoup(r.text, 'html.parser')

### Understanding the HTML strcuture and obtaining a random book info

We are still in the search main page. We will obtain book info, including the sublink.

In [60]:
x = soup.find('table').find_all('tr')[17]

In [61]:
soup.find('table').find_all('tr')

[<tr itemscope="" itemtype="http://schema.org/Book">
 <td valign="top" width="5%">
 <div class="u-anchorTarget" id="25666050"></div>
 <a href="/book/show/25666050-algorithms-to-live-by?from_search=true&amp;from_srp=true&amp;qid=iHFbTUVsHL&amp;rank=1" title="Algorithms to Live By: The Computer Science of Human Decisions">
 <img alt="Algorithms to Live By: The ..." class="bookCover" itemprop="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1454296875i/25666050._SY75_.jpg"/>
 </a> </td>
 <td valign="top" width="100%">
 <a class="bookTitle" href="/book/show/25666050-algorithms-to-live-by?from_search=true&amp;from_srp=true&amp;qid=iHFbTUVsHL&amp;rank=1" itemprop="url">
 <span aria-level="4" itemprop="name" role="heading">Algorithms to Live By: The Computer Science of Human Decisions</span>
 </a> <br/>
 <span class="by">by</span>
 <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
 <div class="authorName__container">
 <a class="authorN

In [72]:
pprint.pprint(x.find('a',{'class':'bookTitle'}).span.text)
pprint.pprint(x.find('a',{'class':'authorName'}).span.text)
pprint.pprint(x.find('span',{'class':'greyText smallText uitext'}).span.text.strip())
pprint.pprint(x.find('a',{'class':'greyText'}).text)

# book sub-link
get_url = x.find('a',{'class':'bookTitle'})
url = get_url.get('href')
pprint.pprint('https://www.goodreads.com{}'.format(url))

# obtains other info for book
y = x.find('span',{'class':'greyText smallText uitext'}).text.replace('\n','')
y = ' '.join(y.split())
y = y.split(' — ')
for i in y:
    pprint.pprint(i)

'Python Programming: An Introduction to Computer Science'
'John Zelle'
'4.02 avg rating — 429 ratings'
'15 editions'
'https://www.goodreads.com/book/show/80440.Python_Programming?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=18'
'4.02 avg rating'
'429 ratings'
'published 2003'
'15 editions'


In [65]:
# create columns strcuture to store data
columns = {'book_title':None,
           'author_name':None,
           'avg_rating':None,
           'rating_count':None,
           'publish year':None,
           'edition':None,
           'link':None}

### Web Scraping Part 1 - Obtain book info from Search Bar Page

You should always click on the second page in the search page. If you in the default page, the search page is without number and you CANNOT do web scraping.

For example:
* [Computer Science Default Page](https://www.goodreads.com/search?utf8=%E2%9C%93&q=Computer+Science&search_type=books) - without page number
* [Computer Science Page 2](https://www.goodreads.com/search?page=2&q=Computer+Science&qid=A44Zff48B9&search_type=books&tab=books&utf8=%E2%9C%93) - has ***page=2&q=Computer+Science*** in the link

* [Mathematics Default Page](https://www.goodreads.com/search?utf8=%E2%9C%93&q=Mathematics&search_type=books)
* [Mathematics Page 2](https://www.goodreads.com/search?page=2&q=Mathematics&qid=tHGardUG6k&search_type=books&tab=books&utf8=%E2%9C%93) - has ***page=2&q=Mathematics***

Then you can use search bar and replace the link inside the code to do simple web scraping

You should:
* replace the url
* change ```for i range(1, 2)``` to ```for i range(1,101)``` 101 means 100 search pages, total 2000 books related to your subject

In [74]:
data = []
for i in range(1, 2): 
    # replace your url with genre or subject that you are interested in 
    api_url = 'https://www.goodreads.com/search?page={}&q=Computer+Science&qid=iHFbTUVsHL&search_type=books&tab=books&utf8=%E2%9C%93'.format(i)
    r = requests.get(api_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # sleep time to 1 second
    time.sleep(1)
    
    # obtain clean data
    for x in soup.find('table').find_all('tr'):
        columns = {}

        columns['book_title'] = x.find('a',{'class':'bookTitle'}).span.text
        columns['author_name'] = x.find('a',{'class':'authorName'}).span.text

        y = x.find('span',{'class':'greyText smallText uitext'}).text.replace('\n','')
        y = ' '.join(y.split())
        y = y.split(' — ')

        if len(y) == 4:
            columns['avg_rating'] = y[0]
            columns['rating_count'] = y[1]
            columns['publish year'] = y[2]
            columns['edition'] = y[3]
        else:
            columns['avg_rating'] = y[0]
            columns['rating_count'] = y[1]
            columns['publish year'] = None
            columns['edition'] = y[2]

        get_url = x.find('a',{'class':'bookTitle'})
        url = get_url.get('href')
        columns['link'] = 'https://www.goodreads.com{}'.format(url)


        data.append(columns)
    

In [75]:
len(data) 

20

In [76]:
# 20 books on the first search page
for i in data:
    print(i)

{'book_title': 'Algorithms to Live By: The Computer Science of Human Decisions', 'author_name': 'Brian Christian', 'avg_rating': '4.14 avg rating', 'rating_count': '21,217 ratings', 'publish year': 'published 2016', 'edition': '46 editions', 'link': 'https://www.goodreads.com/book/show/25666050-algorithms-to-live-by?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=1'}
{'book_title': 'Structure and Interpretation of Computer Programs', 'author_name': 'Harold Abelson', 'avg_rating': '4.46 avg rating', 'rating_count': '4,279 ratings', 'publish year': 'published 1984', 'edition': '23 editions', 'link': 'https://www.goodreads.com/book/show/43713.Structure_and_Interpretation_of_Computer_Programs?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=2'}
{'book_title': 'Concrete Mathematics: A Foundation for Computer Science', 'author_name': 'Ronald L. Graham', 'avg_rating': '4.33 avg rating', 'rating_count': '1,772 ratings', 'publish year': 'published 1988', 'edition': '11 editions', 'link': '

In [77]:
# convert to dataframe
new_data = pd.DataFrame(data)
new_data.head()

,book_title,author_name,avg_rating,rating_count,publish year,edition,link
0,Algorithms to Live By: The Computer Science of...,Brian Christian,4.14 avg rating,"21,217 ratings",published 2016,46 editions,https://www.goodreads.com/book/show/25666050-a...
1,Structure and Interpretation of Computer Programs,Harold Abelson,4.46 avg rating,"4,279 ratings",published 1984,23 editions,https://www.goodreads.com/book/show/43713.Stru...
2,Concrete Mathematics: A Foundation for Compute...,Ronald L. Graham,4.33 avg rating,"1,772 ratings",published 1988,11 editions,https://www.goodreads.com/book/show/112243.Con...
3,Computer Science With C++ Concepts File Handli...,Sumita Arora,4.11 avg rating,555 ratings,None,1 edition,https://www.goodreads.com/book/show/25449409-c...
4,Computer Science With Python (textbook XII),Sumita Arora,3.73 avg rating,481 ratings,None,1 edition,https://www.goodreads.com/book/show/36044968-c...


In [20]:
# create book MongoDB
client = MongoClient('localhost', 27017)
db = client['book']
book_info = db['book_info']

In [35]:
# insert your data
book_info.insert_many(data)

In [36]:
# see how many book stored in your database
book_info.count()

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2060

In [113]:
# print first 5 sublinks of books
for x in book_info.find({},{'_id':False}).limit(5):
    print(x['link'])

https://www.goodreads.com/book/show/533070.Computer_Science?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=21
https://www.goodreads.com/book/show/8266079-multiple-choice-questions-in-computer-science?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=22
https://www.goodreads.com/book/show/28131102-cambridge-international-as-and-a-level-computer-science-revision-guide?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=23
https://www.goodreads.com/book/show/145058.Algorithms_on_Strings_Trees_and_Sequences?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=24
https://www.goodreads.com/book/show/18245828-coding-the-matrix?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=25


### Web Scraping Part 2 - Obtain book info from Sublink Page
After we have obtain the sublink of each book, we can loop through each link and obtain more detailed info for each book.

We will also repeat the step to understand the HTML structure of the sublink to better obtain the book info

In [73]:
# store sublinks into a list
temp_list = []
for x in book_info.find({},{'_id':False}):
    temp_list.append(x['link'])

In [41]:
temp_list[:5]

['https://www.goodreads.com/book/show/533070.Computer_Science?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=21',
 'https://www.goodreads.com/book/show/8266079-multiple-choice-questions-in-computer-science?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=22',
 'https://www.goodreads.com/book/show/28131102-cambridge-international-as-and-a-level-computer-science-revision-guide?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=23',
 'https://www.goodreads.com/book/show/145058.Algorithms_on_Strings_Trees_and_Sequences?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=24',
 'https://www.goodreads.com/book/show/18245828-coding-the-matrix?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=25']

In [115]:
# one sublink
api_url = 'https://www.goodreads.com/book/show/533070.Computer_Science?from_search=true&from_srp=true&qid=iHFbTUVsHL&rank=21'
r = requests.get(api_url)
r.status_code

200

In [116]:
soup = BeautifulSoup(r.text, 'html.parser')

In [117]:
# title
soup.find('h1',{'class':'gr-h1 gr-h1--serif'}).text.strip()

'Computer Science: An Overview'

In [129]:
# author - may more than 1
for x in soup.find_all('div',{'class':'authorName__container'}):
    print(x.find('a',{'class':"authorName"}).span.text)

J. Glenn Brookshear


In [131]:
# use regular expression to find info
import re

In [134]:
# rating
float(soup.find('span',{'itemprop':"ratingValue"}).text.strip())

3.77

In [136]:
# total rating
soup.find('meta',{'itemprop':"ratingCount"}).text.replace('\n','').strip()

'311  ratings'

In [140]:
# rating count Number ONLY
rateCount = 0
for i in (re.findall(r'\d+', soup.find('meta',{'itemprop':"ratingCount"}).text.replace('\n','').strip())):
    rateCount = int(i)
print(rateCount)

311


In [141]:
# review count Number ONLY
reviewCount = 0
for i in (re.findall(r'\d+', soup.find('meta',{'itemprop':"reviewCount"}).text.replace('\n','').strip())):
    reviewCount = int(i)
print(reviewCount)

23


In [143]:
# format
soup.find('span',{'itemprop':"bookFormat"}).text.replace('\n','').strip()

'Paperback'

In [144]:
# number of pages
soup.find('span',{'itemprop':"numberOfPages"}).text.replace('\n','').strip()

'599 pages'

In [145]:
# number of pages Number ONLY
numberOfPages = 0
for i in (re.findall(r'\d+', soup.find('span',{'itemprop':"numberOfPages"}).text.replace('\n','').strip())):
    numberOfPages = int(i)
numberOfPages

599

In [147]:
# language
soup.find('div',{'itemprop':"inLanguage"}).text

'English'

In [148]:
# ISBN
ISBN = ''
for idx, val in enumerate(soup.find('div',{'id':"bookDataBox"}).find_all('div',{'class':'clearFloats'})):
    val = val.text.replace('\n','').strip()
    val = val.replace(' ','')
    if 'ISBN' in val:
        ISBN += val
print(ISBN)

ISBN0321387015(ISBN13:9780321387011)


In [149]:
# data strcuture
second_columns = {'book_title':None,
           'author_name':None,
           'avg_rating':None,
           'rating_count':None,
           'review_count':None,
           'format':None,
           'number_of_page':None,
           'language':None,
           'ISBN':None}

In [151]:
# start new database book detail
db = client['book']
book_detail = db['book_detail']

### Web Scraping Part 2 - Continue...
If you have tried to understand the HTML structure, you may realize that there are some issues. 
Some include:
* book may have more than two authors
* book format not provided
* number of page not provided
* language of book not provided
* ISBN not provided

Therefore, I have included many for try-except block to prevent the for loop crash

You should change `temp_list[:5]` to `temp_list` in the beginning of the `for loop` if you want to obtain all the books!

In [153]:
book_data = []
for x in temp_list[:5]:
    
    # url
    api_url = '{}'.format(x)
    r = requests.get(api_url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # time sleep
    time.sleep(6)
    
    # calling second_columns
    second_columns = {}
    
    
    # title
    second_columns['book_title'] = soup.find('h1',{'class':'gr-h1 gr-h1--serif'}).text.strip()
    
    # author
    if len(soup.find_all('div',{'class':'authorName__container'})) >= 2:
        second_columns['author_name'] = []
        for x in soup.find_all('div',{'class':'authorName__container'}):
            second_columns['author_name'].append(x.find('a',{'class':"authorName"}).span.text)
    else:
        for x in soup.find_all('div',{'class':'authorName__container'}):
            second_columns['author_name'] = x.find('a',{'class':"authorName"}).span.text
       
    # avg rating
    second_columns['avg_rating'] = float(soup.find('span',{'itemprop':"ratingValue"}).text.strip())
    
    rateCount = 0
    for i in (re.findall(r'\d+', soup.find('meta',{'itemprop':"ratingCount"}).text.replace('\n','').strip())):
        rateCount = int(i)
        
    second_columns['rating_count'] = rateCount
    # Below: this line of code is to obtain rating with words such as 311 ratings
    # second_columns['rating_count'] = soup.find('meta',{'itemprop':"ratingCount"}).text.replace('\n','').strip()
    
    # review count
    reviewCount = 0
    for i in (re.findall(r'\d+', soup.find('meta',{'itemprop':"reviewCount"}).text.replace('\n','').strip())):
        reviewCount = int(i)
    
    second_columns['review_count'] = reviewCount
    # Below: this line of code is to obtain rating with words such as 23 reviews
    # second_columns['review_count'] = soup.find('meta',{'itemprop':"reviewCount"}).text.replace('\n','').strip()
    
    # format
    try:
        second_columns['format'] = soup.find('span',{'itemprop':"bookFormat"}).text.replace('\n','').strip()
    except:
        second_columns['format'] = None
    
    # number of pages
    try:   
        numberOfPages = 0
        for i in (re.findall(r'\d+', soup.find('span',{'itemprop':"numberOfPages"}).text.replace('\n','').strip())):
            numberOfPages = int(i)
        second_columns['number_of_page'] = numberOfPages
        # soup.find('span',{'itemprop':"numberOfPages"}).text.replace('\n','').strip()
    except:
        second_columns['number_of_page'] = None

    # language
    try:
        second_columns['language'] = soup.find('div',{'itemprop':"inLanguage"}).text
    except:
        second_columns['language'] = None
    
    # ISBN
    ISBN = ''
    for idx, val in enumerate(soup.find('div',{'id':"bookDataBox"}).find_all('div',{'class':'clearFloats'})):
        val = val.text.replace('\n','').strip()
        val = val.replace(' ','')
        if 'ISBN' in val:
            ISBN += val
    second_columns['ISBN'] = ISBN

    # store data in MongoDB
    book_data.append(second_columns)
    
    # do a search if a book exist
    if len(list(book_detail.find({'book_title':soup.find('h1',{'class':'gr-h1 gr-h1--serif'}).text.strip()}))):
        continue
    else:
        book_detail.insert_one(second_columns)

In [159]:
# search a book in your mongoDB - related to above for loop block
if len(list(book_detail.find({'book_title':'Computer Science With Python (textbook XII)'}))) > 0:
    print('book exists')
else:
    print("book doesn't exist")

book exists


In [155]:
book_data[:5]

[{'book_title': 'Computer Science: An Overview',
  'author_name': 'J. Glenn Brookshear',
  'avg_rating': 3.77,
  'rating_count': 311,
  'review_count': 23,
  'format': 'Paperback',
  'number_of_page': 599,
  'language': 'English',
  'ISBN': 'ISBN0321387015(ISBN13:9780321387011)'},
 {'book_title': 'Multiple Choice Questions in Computer Science',
  'author_name': 'Ela Kumar',
  'avg_rating': 3.62,
  'rating_count': 66,
  'review_count': 9,
  'format': 'Paperback',
  'number_of_page': 472,
  'language': None,
  'ISBN': 'ISBN819067577X(ISBN13:9788190675772)'},
 {'book_title': 'Cambridge International as and a Level Computer Science Revision Guide',
  'author_name': 'Tony Piper',
  'avg_rating': 4.29,
  'rating_count': 41,
  'review_count': 4,
  'format': 'Paperback',
  'number_of_page': 328,
  'language': None,
  'ISBN': 'ISBN1107547547(ISBN13:9781107547544)'},
 {'book_title': 'Algorithms on Strings, Trees and Sequences: Computer Science and Computational Biology',
  'author_name': 'Dan Gu

In [160]:
book_detail.estimated_document_count()

1902

In [161]:
list(book_detail.find({},{'_id':False}).limit(10))

[{'book_title': 'Computer Science: An Overview',
  'author_name': 'J. Glenn Brookshear',
  'avg_rating': 3.76,
  'rating_count': 310,
  'review_count': 22,
  'format': 'Paperback',
  'number_of_page': 599,
  'language': 'English',
  'ISBN': 'ISBN0321387015(ISBN13:9780321387011)'},
 {'book_title': 'Multiple Choice Questions in Computer Science',
  'author_name': 'Ela Kumar',
  'avg_rating': 3.62,
  'rating_count': 66,
  'review_count': 9,
  'format': 'Paperback',
  'number_of_page': 472,
  'language': None,
  'ISBN': 'ISBN819067577X(ISBN13:9788190675772)'},
 {'book_title': 'Cambridge International as and a Level Computer Science Revision Guide',
  'author_name': 'Tony Piper',
  'avg_rating': 4.29,
  'rating_count': 41,
  'review_count': 4,
  'format': 'Paperback',
  'number_of_page': 328,
  'language': None,
  'ISBN': 'ISBN1107547547(ISBN13:9781107547544)'},
 {'book_title': 'Algorithms on Strings, Trees and Sequences: Computer Science and Computational Biology',
  'author_name': 'Dan Gu

In [162]:
df = pd.DataFrame(book_detail.find({},{'_id':False}))

In [163]:
df.head()

,book_title,author_name,avg_rating,rating_count,review_count,format,number_of_page,language,ISBN
0,Computer Science: An Overview,J. Glenn Brookshear,3.76,310,22,Paperback,599.0,English,ISBN0321387015(ISBN13:9780321387011)
1,Multiple Choice Questions in Computer Science,Ela Kumar,3.62,66,9,Paperback,472.0,None,ISBN819067577X(ISBN13:9788190675772)
2,Cambridge International as and a Level Compute...,Tony Piper,4.29,41,4,Paperback,328.0,None,ISBN1107547547(ISBN13:9781107547544)
3,"Algorithms on Strings, Trees and Sequences: Co...",Dan Gusfield,4.09,85,4,Hardcover,556.0,English,ISBN0521585198(ISBN13:9780521585194)
4,Coding the Matrix: Linear Algebra through Comp...,Philip N. Klein,4.23,115,5,Paperback,528.0,English,ISBN061585673X(ISBN13:9780615856735)


In [166]:
df.tail(5)

,book_title,author_name,avg_rating,rating_count,review_count,format,number_of_page,language,ISBN
1897,Algorithms And Models For The Web Graph: 6th I...,"[Konstantin Avrachenkov, Debora Donato, Nelly ...",0.0,0,0,Paperback,183.0,English,ISBN3540959947(ISBN13:9783540959946)
1898,International E-Conference on Computer Science...,"[E. Simos, Theodore Simos, G. G. Psihoyios]",0.0,0,0,Paperback,196.0,English,ISBN9067644250(ISBN13:9789067644259)
1899,Transactions On Data Hiding And Multimedia Sec...,Yun Q. Shi,0.0,0,0,Paperback,129.0,English,ISBN3642142974(ISBN13:9783642142970)
1900,Optical Supercomputing: Second International W...,"[Shlomi Dolev, Mihai Oltean]",0.0,0,0,Paperback,153.0,English,ISBN364210441X(ISBN13:9783642104411)
1901,Mathematical Foundations of Computer Science 2...,"[Petr Hliněný, Antonín Kučera]",0.0,0,0,Paperback,714.0,English,ISBN364215154X(ISBN13:9783642151545)


In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   book_title      1902 non-null   object 
 1   author_name     1902 non-null   object 
 2   avg_rating      1902 non-null   float64
 3   rating_count    1902 non-null   int64  
 4   review_count    1902 non-null   int64  
 5   format          1852 non-null   object 
 6   number_of_page  1573 non-null   float64
 7   language        956 non-null    object 
 8   ISBN            1902 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 133.9+ KB


In [165]:
df.describe()

,avg_rating,rating_count,review_count,number_of_page
count,1902.000000,1902.000000,1902.000000,1573.000000
mean,2.109311,10.786015,1.182440,387.778767
std,2.017028,61.096563,9.845778,305.735283
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,189.000000
50%,2.790000,1.000000,0.000000,332.000000
75%,4.000000,3.000000,0.000000,512.000000
max,5.000000,965.000000,218.000000,3348.000000
